In [1]:
import pandas as pd
import numpy as np
import glob
import csv
from collections import defaultdict
import re
import json

In [2]:
election = pd.read_csv("US_County_Level_Presidential_Results_08-16.csv")

In [3]:
election.dropna(axis=1, inplace=True)

## Collecting Census Data

In [4]:
import urllib.request
import ast


class Census:
    def __init__(self, key):
        self.key = key

    def get(self, fields, geo, year=2012, dataset='sf1'):
        fields = [','.join(fields)]
        base_url = 'http://api.census.gov/data/%s/%s?key=%s&get=' % (str(year), dataset, self.key)
        query = fields
        for item in geo:
            query.append(item)
        add_url = '&'.join(query)
        url = base_url + add_url
        print(url)
        req = urllib.request.Request(url)
        response = urllib.request.urlopen(req)
        return response.read()

In [5]:
census = Census('6c3fba83f5ccd2adb2fd3894045431e20e3ec5cb')

In [6]:
fieldsToGet = dict({
    ('ST', 'State FIPS'),
    ('COUNTY', 'County FIPS Code'),
    ('NAME', 'County Name'),
    ('B00002_001E', '# of Housing Units'),
    ('B01001_001E', 'Population'),
    ('B01001_002E', '# Male'),
    ('B01001_026E', '# Female'),
    ('B01001A_001E', '# White'),
    ('B01001A_002E', '# White Male'),
    ('B01001A_017E', '# White Female'),
    ('B01001B_001E', '# Black'),
    ('B01001B_002E', '# Black Male'),
    ('B01001B_017E', '# Black Female'),
    ('B01001H_001E', '# White Not Hispanic'),
    ('B01001H_002E', '# White Not Hispanis Male'),
    ('B01001H_017E', '# White Not Hispanis Female'),
    ('B01001I_001E', '# Hispanic'),
    ('B01001I_002E', '# Hispanic Male'),
    ('B01001I_017E', '# Hispanic Female'),
    ('B01002_001E', 'Median Age'),
    ('B01002_002E', 'Median Age Male'),
    ('B01002_003E', 'Median Age Female'),
    ('B05001_002E', 'Citizen, born in US'),
    ('B05001_005E', 'Citizen by Naturalization'),
    ('B05002_014E', 'Foreign born naturalized'),
    ('B06008_002E', 'Never Married'),
    ('B06008_003E', 'Currently Married'),
    ('B06008_004E', 'Divorced'),
    ('B06008_005E', 'Separated'),
    ('B06009_001E', 'Education Count'),
    ('B06009_002E', 'Less Than HS'),
    ('B06009_003E', 'HS or equal'),
    ('B06009_004E', 'Some College-AA'),
    ('B06009_005E', 'Bachelor'),
    ('B06009_006E', 'Graduate'),
    ('B06009_007E', 'Total Born In State education'),
    ('B06009_008E', 'In State-Less than HS'),
    ('B06009_009E', 'In State-HS or equal'),
    ('B06009_010E', 'In State-Some College'),
    ('B06009_011E', 'In State-Bachelor'),
    ('B06009_012E', 'In State-Graduate'),
    ('B06009_013E', 'Born in Other State Education'),
    ('B06009_014E', 'Other State-Less than HS'),
    ('B06009_015E', 'Other State-HS or equal'),
    ('B06009_016E', 'Other State-Some College'),
    ('B06009_017E', 'Other State-Bachelor'),
    ('B06009_018E', 'Other State-Graduate'),
    ('B06010_001E', 'Total Reporting Income'),
    ('B06010_002E', 'Total Reporing 0 Income'),
    ('B06010_003E', 'Total Reporting with Income'),
})

fieldsToGet2 = dict({
    ('ST', 'State FIPS'),
    ('COUNTY', 'County FIPS Code'),
    ('NAME', 'County Name'),
    ('B06010_004E', 'Income < 10K'),
    ('B06010_005E', 'Income 10-15K'),
    ('B06010_006E', 'Income 15-25K'),
    ('B06010_007E', 'Income 25-35K'),
    ('B06010_008E', 'Income 35-50K'),
    ('B06010_009E', 'Income 50-65K'),
    ('B06010_010E', 'Income 65-75K'),
    ('B06010_011E', 'Income > 75K'),
    ('B06011_001E', 'Median Income'),
    ('B06011_002E', 'Median Income Born In State'),
    ('B06011_003E', 'Median Income Born Other State'),
    ('B06011_004E', 'Median Income Born Outside US, Native'),
    ('B06011_005E', 'Median Income Foreign Born'),
    ('B06012_001E', 'Total In Poverty Pool'),
    ('B06012_002E', '100% below povery line'),
    ('B06012_003E', '1-1.5X of poverty line'),
    ('B06012_004E', 'At above 150% poverty line'),
    ('B06012_006E', 'Born In State below 100% poverty'),
    ('B06012_007E', 'Born In State 1-1.5X of poverty line'),
    ('B06012_008E', 'Born In State at/above 1.5X poverty line'),
    ('B06012_010E', 'Born Other State below 100% poverty'),
    ('B06012_011E', 'Born Other State 1-1.5X of poverty line'),
    ('B06012_012E', 'Born Other State at/above 1.5X poverty line'),
    ('B06012_018E', 'Foreign Born below 100% poverty'),
    ('B06012_019E', 'Foreign Born 1-1.5X of poverty line'),
    ('B06012_020E', 'Foreign Born at/above 1.5X poverty line'),
    ('B07001_001E', 'Total Moving'),
    ('B07001_017E', 'Same house 1 year ago'),
    ('B07001_033E', 'Moved Within County'),
    ('B07001_049E', 'Moved Within Same State'),
    ('B07001_065E', 'Moved From Different State'),
    ('B07001_081E', 'Moved From Abroad'),
    ('B07004A_001E', 'Total Moving White'),
    ('B07004A_002E', 'Same house 1 year ago White'),
    ('B07004A_003E', 'Moved Within County White'),
    ('B07004A_004E', 'Moved Within Same State White'),
    ('B07004A_005E', 'Moved From Different State White'),
    ('B07004A_006E', 'Moved From Abroad White')
    })

fieldsToGet3 = dict({
    ('ST', 'State FIPS'),
    ('COUNTY', 'County FIPS Code'),
    ('NAME', 'County Name'),
    ('B19101_002E', 'Family Income < 10K'),
    ('B19101_003E', 'Family Income 10-15K'),
    ('B19101_004E', 'Family Income 15-20K'),
    ('B19101_004E', 'Family Income 20-25K'),
    ('B19101_006E', 'Family Income 25-30K'),
    ('B19101_007E', 'Family Income 30-35K'),
    ('B19101_008E', 'Family Income 35-40K'),
    ('B19101_009E', 'Family Income 40-45K'),
    ('B19101_010E', 'Family Income 45-50K'),
    ('B19101_011E', 'Family Income 50-60K'),
    ('B19101_012E', 'Family Income 60-75K'),
    ('B19101_013E', 'Family Income 75-100K'),
    ('B19101_014E', 'Family Income 100-125K'),
    ('B19101_015E', 'Family Income 125-150K'),
    ('B19101_016E', 'Family Income 150-200K'),
    ('B19101_017E', 'Family Income > 200K'),
    ('B08006_001E', 'Total Means of Transportation Work'),
    ('B08006_002E', 'Work by car, truck, van'),
    ('B08006_003E', 'Drive Alone'),
    ('B08006_004E', 'Car pooled'),
    ('B08006_008E', 'Public Transportation'),
    ('B08006_012E', 'Rail Road'),
    ('B08006_013E', 'Ferry Boat'),
    ('B08006_014E', 'Bicycle'),
    ('B08006_015E', 'Walked'),
    ('B08006_017E', 'Work at Home'),
    ('B08007_002E', 'Work in State'),
    ('B08007_003E', 'Work in County'),
    ('B08007_004E', 'Work outside County'),
    ('B08007_005E', 'Work outside State'),
    ('B08014_002E', 'No Vehicle Owned'),
    ('B08014_003E', '1 vehicle'),
    ('B08014_004E', '2 vehicles'),
    ('B08014_005E', '3 vehicles'),
    ('B08014_006E', '4 vehicles'),
    ('B08014_007E', '5 or more vehicles'),
    })

# CHANGE ME IF YOU WANT DATA FROM DIFFERENT YEARS ONY 2011 - 2015 though

In [67]:
year = 2014

In [68]:
AllCounty = census.get(list(fieldsToGet.keys()), 
                   ['for=county:*', 'in=state:*'], 
                   year=year, 
                   dataset='acs5')
AllCounty2 = census.get(list(fieldsToGet2.keys()), 
                   ['for=county:*', 'in=state:*'], 
                   year=year, 
                   dataset='acs5')
AllCounty3 = census.get(list(fieldsToGet3.keys()),
                 ['for=county:*', 'in=state:*'], 
                   year=year, 
                   dataset='acs5')

http://api.census.gov/data/2014/acs5?key=6c3fba83f5ccd2adb2fd3894045431e20e3ec5cb&get=B01001H_002E,B06010_002E,B06009_016E,B00002_001E,B06010_001E,B05002_014E,B06009_018E,B06009_013E,B06009_004E,B01001A_017E,B01001H_001E,B01001_002E,B06010_003E,B06008_005E,B06009_006E,B06009_017E,B06009_014E,B01001I_001E,B01001I_002E,B01002_002E,B06008_004E,B01001H_017E,B06009_012E,B06009_001E,B01001_026E,B06009_011E,B05001_005E,B06009_007E,B01002_001E,COUNTY,B01001A_002E,B06008_002E,B06009_003E,B01001A_001E,B01001B_017E,B06009_005E,NAME,B06009_008E,B06008_003E,B01001I_017E,B06009_015E,B01001B_002E,B06009_002E,B01001B_001E,ST,B01002_003E,B06009_009E,B06009_010E,B05001_002E,B01001_001E&for=county:*&in=state:*
http://api.census.gov/data/2014/acs5?key=6c3fba83f5ccd2adb2fd3894045431e20e3ec5cb&get=B06011_004E,B06012_002E,B06010_008E,B06012_007E,B06012_012E,B07001_049E,COUNTY,B06012_020E,B06010_009E,B07004A_006E,B06012_019E,B06012_018E,B07004A_003E,B07001_001E,B07001_033E,B06012_011E,B06010_005E,B07004A_001E

In [69]:
AllCounty = str(AllCounty)
splits = AllCounty.split(']')

#Making dict to store all values
fields = splits[0][5:].split('","')
fields1 = {}
for key in fields:
    fields1['%s' % key] = []

In [70]:
#parsing out line to add to fields1 dict, then convert to df
for line in splits[1:]:
    try:
        line = line.split('[')[1]
        line = line.split('null')
        if len(line) > 1: # skip entries with null
            pass
        elements = line[0].split('","')
        if (len(elements) == len(fields1.keys())):
            for field, element in zip(fields, elements):
                fields1['%s' % field].append(element.strip('"'))
    except:
        pass

In [71]:
AllCounty2 = str(AllCounty2)
splits = AllCounty2.split(']')

#Making dict to store all values
fields = splits[0][5:].split('","')
fields2 = {}
for key in fields:
    fields2['%s' % key] = []

In [72]:
#parsing out line to add to fields1 dict, then convert to df
for line in splits[1:]:
    try:
        line = line.split('[')[1]
        line = line.split('null')
        if len(line) > 1: # skip entries with null
            pass
        elements = line[0].split('","')
        if (len(elements) == len(fields2.keys())):
            for field, element in zip(fields, elements):
                fields2['%s' % field].append(element.strip('"'))
    except:
        pass

In [73]:
AllCounty3 = str(AllCounty3)
splits = AllCounty3.split(']')

#Making dict to store all values
fields = splits[0][5:].split('","')
fields3 = {}
for key in fields:
    fields3['%s' % key] = []

In [74]:
#parsing out line to add to fields1 dict, then convert to df
for line in splits[1:]:
    try:
        line = line.split('[')[1]
        line = line.split('null')
        if len(line) > 1: # skip entries with null
            pass
        elements = line[0].split('","')
        if (len(elements) == len(fields3.keys())):
            for field, element in zip(fields, elements):
                fields3['%s' % field].append(element.strip('"'))
    except:
        pass

Merging the Data frames

In [75]:
df = pd.DataFrame(fields1)
df2 = pd.DataFrame(fields2)
df3 = pd.DataFrame(fields3)

In [76]:
print(df.shape)
print(df2.shape)
print(df3.shape)

(3142, 52)
(2655, 43)
(3220, 40)


In [77]:
finalDf = pd.merge(pd.merge(df, df2, on='NAME'), df3, on='NAME')

In [78]:
finalDf['fips_code'] = (finalDf['ST'] + finalDf['COUNTY']).astype('int')

In [79]:
finalDf = pd.merge(election, finalDf, on='fips_code')

# DATAFRAME TO USE

# Data Cleaning

In [80]:
finalDf.isnull().sum()

fips_code       0
county          0
total_2008      0
dem_2008        0
gop_2008        0
oth_2008        0
total_2012      0
dem_2012        0
gop_2012        0
oth_2012        0
total_2016      0
dem_2016        0
gop_2016        0
oth_2016        0
B00002_001E     0
B01001A_001E    0
B01001A_002E    0
B01001A_017E    0
B01001B_001E    0
B01001B_002E    0
B01001B_017E    0
B01001H_001E    0
B01001H_002E    0
B01001H_017E    0
B01001I_001E    0
B01001I_002E    0
B01001I_017E    0
B01001_001E     0
B01001_002E     0
B01001_026E     0
               ..
B08006_017E     0
B08007_002E     0
B08007_003E     0
B08007_004E     0
B08007_005E     0
B08014_002E     0
B08014_003E     0
B08014_004E     0
B08014_005E     0
B08014_006E     0
B08014_007E     0
B19101_002E     0
B19101_003E     0
B19101_004E     0
B19101_006E     0
B19101_007E     0
B19101_008E     0
B19101_009E     0
B19101_010E     0
B19101_011E     0
B19101_012E     0
B19101_013E     0
B19101_014E     0
B19101_015E     0
B19101_016

In [81]:
for i in zip(finalDf.columns, finalDf.iloc[0]):
    print(i)

('fips_code', 26041)
('county', 'Delta County')
('total_2008', 19064)
('dem_2008', 9974)
('gop_2008', 8763)
('oth_2008', 327)
('total_2012', 18043)
('dem_2012', 8330)
('gop_2012', 9533)
('oth_2012', 180)
('total_2016', 18467)
('dem_2016', 6431)
('gop_2016', 11112)
('oth_2016', 924)
('B00002_001E', '3040')
('B01001A_001E', '34754')
('B01001A_002E', '17231')
('B01001A_017E', '17523')
('B01001B_001E', '121')
('B01001B_002E', '60')
('B01001B_017E', '61')
('B01001H_001E', '34581')
('B01001H_002E', '17137')
('B01001H_017E', '17444')
('B01001I_001E', '362')
('B01001I_002E', '153')
('B01001I_017E', '209')
('B01001_001E', '36841')
('B01001_002E', '18235')
('B01001_026E', '18606')
('B01002_001E', '46.3')
('B01002_002E', '45.1')
('B01002_003E', '47.3')
('B05001_002E', '36323')
('B05001_005E', '260')
('B05002_014E', '260')
('B06008_002E', '7509')
('B06008_003E', '16389')
('B06008_004E', '3981')
('B06008_005E', '379')
('B06009_001E', '26625')
('B06009_002E', '2342')
('B06009_003E', '9770')
('B06009

In [82]:
from collections import Counter

In [83]:
Counter(finalDf[finalDf["B00002_001E"].isnull()]["NAME"].apply(lambda x: x.split(", ")[1]))

Counter()

In [84]:
finalDf = finalDf.drop(["COUNTY_x", "NAME", "ST_x", "state_x", 'COUNTY', 'ST', 'county"', 'COUNTY_y', 'state', 'ST_y', 'county"_y','state_y'], axis=1)

In [85]:
del finalDf['county"_x']

In [86]:
finalDf = finalDf.dropna().reset_index(drop=True)

In [87]:
finalDf.shape

(2628, 134)

In [88]:
finalDf.head()

,fips_code,county,total_2008,dem_2008,gop_2008,oth_2008,total_2012,dem_2012,gop_2012,oth_2012,...,B19101_008E,B19101_009E,B19101_010E,B19101_011E,B19101_012E,B19101_013E,B19101_014E,B19101_015E,B19101_016E,B19101_017E
0,26041,Delta County,19064,9974,8763,327,18043,8330,9533,180,...,453,572,528,1217,1261,1697,623,401,234,96
1,48295,Lipscomb County,1256,155,1093,8,1168,119,1044,5,...,20,29,33,79,123,116,91,54,56,46
2,1127,Walker County,28652,7420,20722,510,28497,6551,21633,313,...,1041,919,987,1570,2209,1819,1210,745,437,339
3,48389,Reeves County,3077,1606,1445,26,2867,1649,1185,33,...,96,119,75,213,320,350,188,69,64,79
4,56017,Hot Springs County,2546,619,1834,93,2495,523,1894,78,...,50,69,84,122,234,112,148,89,36,58


In [89]:
for i in zip(finalDf.columns, finalDf.iloc[0]):
    print(i)

('fips_code', 26041)
('county', 'Delta County')
('total_2008', 19064)
('dem_2008', 9974)
('gop_2008', 8763)
('oth_2008', 327)
('total_2012', 18043)
('dem_2012', 8330)
('gop_2012', 9533)
('oth_2012', 180)
('total_2016', 18467)
('dem_2016', 6431)
('gop_2016', 11112)
('oth_2016', 924)
('B00002_001E', '3040')
('B01001A_001E', '34754')
('B01001A_002E', '17231')
('B01001A_017E', '17523')
('B01001B_001E', '121')
('B01001B_002E', '60')
('B01001B_017E', '61')
('B01001H_001E', '34581')
('B01001H_002E', '17137')
('B01001H_017E', '17444')
('B01001I_001E', '362')
('B01001I_002E', '153')
('B01001I_017E', '209')
('B01001_001E', '36841')
('B01001_002E', '18235')
('B01001_026E', '18606')
('B01002_001E', '46.3')
('B01002_002E', '45.1')
('B01002_003E', '47.3')
('B05001_002E', '36323')
('B05001_005E', '260')
('B05002_014E', '260')
('B06008_002E', '7509')
('B06008_003E', '16389')
('B06008_004E', '3981')
('B06008_005E', '379')
('B06009_001E', '26625')
('B06009_002E', '2342')
('B06009_003E', '9770')
('B06009

In [90]:
finalDf = pd.concat([finalDf.ix[:, :2], finalDf[finalDf.columns[2:]].applymap(float)], axis=1)

In [91]:
from mapping import mapping

In [92]:
finalDf = finalDf.rename(columns=mapping)

In [93]:
#there might be some NAs in the demographics field, not sure what we want to do about them
#drop the county? what if it's an important county?
finalDf.head()

,fips_code,county,total_2008,dem_2008,gop_2008,oth_2008,total_2012,dem_2012,gop_2012,oth_2012,...,Family Income 35-40K,Family Income 40-45K,Family Income 45-50K,Family Income 50-60K,Family Income 60-75K,Family Income 75-100K,Family Income 100-125K,Family Income 125-150K,Family Income 150-200K,Family Income > 200K
0,26041,Delta County,19064.0,9974.0,8763.0,327.0,18043.0,8330.0,9533.0,180.0,...,453.0,572.0,528.0,1217.0,1261.0,1697.0,623.0,401.0,234.0,96.0
1,48295,Lipscomb County,1256.0,155.0,1093.0,8.0,1168.0,119.0,1044.0,5.0,...,20.0,29.0,33.0,79.0,123.0,116.0,91.0,54.0,56.0,46.0
2,1127,Walker County,28652.0,7420.0,20722.0,510.0,28497.0,6551.0,21633.0,313.0,...,1041.0,919.0,987.0,1570.0,2209.0,1819.0,1210.0,745.0,437.0,339.0
3,48389,Reeves County,3077.0,1606.0,1445.0,26.0,2867.0,1649.0,1185.0,33.0,...,96.0,119.0,75.0,213.0,320.0,350.0,188.0,69.0,64.0,79.0
4,56017,Hot Springs County,2546.0,619.0,1834.0,93.0,2495.0,523.0,1894.0,78.0,...,50.0,69.0,84.0,122.0,234.0,112.0,148.0,89.0,36.0,58.0


In [94]:
import pickle


In [ ]:
f = open("./final_data2011.p", "wb")
pickle.dump(finalDf, f)

In [95]:
f = open("./final_data.p", "wb")

pickle.dump(finalDf, f)

Try and predict change in number of voters over the years. 

Try and predict the change in demographic voting ratio. 

The project sheet said we have to use some clustering techniques to see which county was mis-clustered. Not sure if this is the best way to predict tho..

Also the sheet mentioned about building models that will explain things, so I was thinking of using Lasso or elastic net, and we can pull census data from 2011 and 2015 to try to see which variable were the most important features we from each data set

In [96]:
finalDf

,fips_code,county,total_2008,dem_2008,gop_2008,oth_2008,total_2012,dem_2012,gop_2012,oth_2012,...,Family Income 35-40K,Family Income 40-45K,Family Income 45-50K,Family Income 50-60K,Family Income 60-75K,Family Income 75-100K,Family Income 100-125K,Family Income 125-150K,Family Income 150-200K,Family Income > 200K
0,26041,Delta County,19064.0,9974.0,8763.0,327.0,18043.0,8330.0,9533.0,180.0,...,453.0,572.0,528.0,1217.0,1261.0,1697.0,623.0,401.0,234.0,96.0
1,48295,Lipscomb County,1256.0,155.0,1093.0,8.0,1168.0,119.0,1044.0,5.0,...,20.0,29.0,33.0,79.0,123.0,116.0,91.0,54.0,56.0,46.0
2,1127,Walker County,28652.0,7420.0,20722.0,510.0,28497.0,6551.0,21633.0,313.0,...,1041.0,919.0,987.0,1570.0,2209.0,1819.0,1210.0,745.0,437.0,339.0
3,48389,Reeves County,3077.0,1606.0,1445.0,26.0,2867.0,1649.0,1185.0,33.0,...,96.0,119.0,75.0,213.0,320.0,350.0,188.0,69.0,64.0,79.0
4,56017,Hot Springs County,2546.0,619.0,1834.0,93.0,2495.0,523.0,1894.0,78.0,...,50.0,69.0,84.0,122.0,234.0,112.0,148.0,89.0,36.0,58.0
5,37183,Wake County,442245.0,250891.0,187001.0,4353.0,526805.0,286939.0,232933.0,6933.0,...,7325.0,7586.0,8757.0,16602.0,22785.0,33803.0,28914.0,20772.0,24173.0,23019.0
6,37147,Pitt County,74884.0,40501.0,33927.0,456.0,76814.0,40701.0,35534.0,579.0,...,1663.0,1867.0,1689.0,3630.0,4223.0,5463.0,3918.0,1896.0,1665.0,1545.0
7,48497,Wise County,20639.0,4471.0,15973.0,195.0,20692.0,3219.0,17178.0,295.0,...,696.0,967.0,712.0,1594.0,1653.0,2605.0,1753.0,987.0,903.0,611.0
8,5053,Grant County,6793.0,1562.0,5023.0,208.0,6478.0,1467.0,4829.0,182.0,...,202.0,161.0,269.0,516.0,581.0,862.0,506.0,141.0,152.0,67.0
9,31147,Richardson County,3968.0,1513.0,2342.0,113.0,3680.0,1180.0,2407.0,93.0,...,88.0,117.0,109.0,185.0,274.0,395.0,202.0,69.0,66.0,48.0
